In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from collections import defaultdict
import os, sys
sys.path.insert(0, "/mnt/d/dev/git/miRExplore/python/")


import scispacy
import spacy
from spacy import displacy

nlp = spacy.load('/mnt/f/spacy/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')
nlp_ent = spacy.load("/mnt/f/spacy/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")


In [ ]:
! ls /mnt/f/spacy/en_ner_bionlp13cg_md-0.2.5/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.5

In [18]:
%autoreload 2
import textmining.MirGeneRelCheck
from intervaltree import Interval, IntervalTree

In [12]:

def analyse(testCase, dist=80):
    relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck(nlp_ent)

    doc = nlp(testCase[0])
    entDoc = nlp_ent(testCase[0])
    
    enttree = IntervalTree()
    for ent in entDoc.ents:
        enttree.addi(ent.start_char, ent.end_char, (ent.label_, ent.text))

    
    lWord = doc[testCase[1]]
    rWord = doc[testCase[2]]

    for t in doc:
        
        possTokenEnts = enttree.overlap(t.idx, t.idx+len(t.text))
        ents = set([x.data for x in possTokenEnts])
        
        print(t.idx, t.idx+len(t.text), t.i, t, t.pos_, t.dep_, t.head, t.head.idx, ents)

    print(lWord, rWord)

    if not "mir" in str(lWord).lower() and not "let" in str(lWord).lower():
        tmp = lWord
        lWord = rWord
        rWord = tmp
        
    print(lWord, rWord)
    
    checkResults = relCheck.checkRelation(doc, lWord, rWord, verbose=True)
    print(checkResults)
    
    displacy.render(doc, style="dep", options={"compact": True, "distance": dist})
    
    entDoc = nlp_ent(testCase[0])
    
    
    displacy.render(entDoc, style="ent", options={"compact": True, "distance": dist})
    
    return enttree

In [ ]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Functional association between miR-124a and CDK6 in medulloblastoma was established using luciferase assays.", 4, 6)

enttree = analyse(testcase)

In [ ]:
[x.data for x in enttree.overlap(31,39)]

In [6]:
analyse(
(
    "miR-10a inhibits dendritic cell activation and Th1/Th17 cell immune responses in IBD.",
    0,
    6
)
)

0 7 0 miR-10a NOUN nsubj inhibits 8 {('GENE_OR_GENE_PRODUCT', 'miR-10a')}
8 16 1 inhibits VERB ROOT inhibits 8 set()
17 26 2 dendritic ADJ amod activation 32 {('CELL', 'dendritic cell')}
27 31 3 cell NOUN compound activation 32 {('CELL', 'dendritic cell')}
32 42 4 activation NOUN dobj inhibits 8 set()
43 46 5 and CCONJ cc activation 32 set()
47 55 6 Th1/Th17 NOUN compound responses 68 set()
56 60 7 cell NOUN compound responses 68 {('CELL', 'cell')}
61 67 8 immune ADJ amod responses 68 set()
68 77 9 responses NOUN conj activation 32 set()
78 80 10 in ADP case IBD 81 set()
81 84 11 IBD NOUN nmod activation 32 set()
84 85 12 . PUNCT punct inhibits 8 set()
miR-10a Th1/Th17
miR-10a Th1/Th17
Conjuncts [responses, activation]
neighbour responses in
neighbour responses in
Conjunction before fc activation
fc add children for activation
fc add children for activation
Conjuncts [activation, responses]
neighbour activation and
neighbour activation and
Conjunction before fc responses
fc add childre

IntervalTree([Interval(0, 7, ('GENE_OR_GENE_PRODUCT', 'miR-10a')), Interval(17, 31, ('CELL', 'dendritic cell')), Interval(56, 60, ('CELL', 'cell'))])

In [7]:
analyse(
(
    "miR-10a inhibits dendritic cell activation and Th1/Th17 cell immune responses in IBD.",
    0,
    6
)
)

0 7 0 miR-10a NOUN nsubj inhibits 8 {('GENE_OR_GENE_PRODUCT', 'miR-10a')}
8 16 1 inhibits VERB ROOT inhibits 8 set()
17 26 2 dendritic ADJ amod activation 32 {('CELL', 'dendritic cell')}
27 31 3 cell NOUN compound activation 32 {('CELL', 'dendritic cell')}
32 42 4 activation NOUN dobj inhibits 8 set()
43 46 5 and CCONJ cc activation 32 set()
47 55 6 Th1/Th17 NOUN compound responses 68 set()
56 60 7 cell NOUN compound responses 68 {('CELL', 'cell')}
61 67 8 immune ADJ amod responses 68 set()
68 77 9 responses NOUN conj activation 32 set()
78 80 10 in ADP case IBD 81 set()
81 84 11 IBD NOUN nmod activation 32 set()
84 85 12 . PUNCT punct inhibits 8 set()
miR-10a Th1/Th17
miR-10a Th1/Th17
Conjuncts [responses, activation]
neighbour responses in
neighbour responses in
Conjunction before fc activation
fc add children for activation
fc add children for activation
Conjuncts [activation, responses]
neighbour activation and
neighbour activation and
Conjunction before fc responses
fc add childre

IntervalTree([Interval(0, 7, ('GENE_OR_GENE_PRODUCT', 'miR-10a')), Interval(17, 31, ('CELL', 'dendritic cell')), Interval(56, 60, ('CELL', 'cell'))])

In [8]:
analyse(
(
    "Mir-29b mediates the regulation of Nrf2 on airway epithelial remodeling and Th1/Th2 differentiation in COPD rats",
    0,
    11
)
)

0 7 0 Mir-29b NOUN nsubj mediates 8 {('GENE_OR_GENE_PRODUCT', 'Mir-29b')}
8 16 1 mediates VERB ROOT mediates 8 set()
17 20 2 the DET det regulation 21 set()
21 31 3 regulation NOUN dobj mediates 8 set()
32 34 4 of ADP case Nrf2 35 set()
35 39 5 Nrf2 NOUN nmod regulation 21 {('GENE_OR_GENE_PRODUCT', 'Nrf2')}
40 42 6 on ADP case remodeling 61 set()
43 49 7 airway NOUN compound remodeling 61 {('TISSUE', 'airway epithelial')}
50 60 8 epithelial ADJ amod remodeling 61 {('TISSUE', 'airway epithelial')}
61 71 9 remodeling NOUN nmod regulation 21 set()
72 75 10 and CCONJ cc remodeling 61 set()
76 83 11 Th1/Th2 NOUN compound differentiation 84 set()
84 99 12 differentiation NOUN conj remodeling 61 set()
100 102 13 in ADP case rats 108 set()
103 107 14 COPD NOUN compound rats 108 set()
108 112 15 rats NOUN nmod remodeling 61 {('ORGANISM', 'rats')}
Mir-29b Th1/Th2
Mir-29b Th1/Th2
Conjuncts [differentiation, remodeling]
neighbour differentiation in
neighbour differentiation in
Conjunction before f

IntervalTree([Interval(0, 7, ('GENE_OR_GENE_PRODUCT', 'Mir-29b')), Interval(35, 39, ('GENE_OR_GENE_PRODUCT', 'Nrf2')), Interval(43, 60, ('TISSUE', 'airway epithelial')), Interval(108, 112, ('ORGANISM', 'rats'))])

In [27]:
analyse(
(
    "Here we show that the microRNA- (miR-) 31 is upregulated in murine Th1 cells with a history of repeated reactivation and in memory Th cells isolated from the synovial fluid of patients with rheumatic joint disease",
    14,
    5
)
)

0 4 0 Here ADV advmod show 8 set()
5 7 1 we PRON nsubj show 8 set()
8 12 2 show VERB ROOT show 8 set()
13 17 3 that SCONJ mark upregulated 45 set()
18 21 4 the DET det microRNA- 22 set()
22 31 5 microRNA- NOUN nsubjpass upregulated 45 {('GENE_OR_GENE_PRODUCT', 'microRNA-')}
32 33 6 ( PUNCT punct miR- 33 set()
33 37 7 miR- NOUN appos microRNA- 22 set()
37 38 8 ) PUNCT punct miR- 33 set()
39 41 9 31 NUM nummod microRNA- 22 set()
42 44 10 is AUX auxpass upregulated 45 set()
45 56 11 upregulated VERB ccomp show 8 set()
57 59 12 in ADP case cells 71 set()
60 66 13 murine ADJ amod cells 71 {('ORGANISM', 'murine Th1 cells')}
67 70 14 Th1 NOUN compound cells 71 {('ORGANISM', 'murine Th1 cells')}
71 76 15 cells NOUN nmod upregulated 45 {('ORGANISM', 'murine Th1 cells')}
77 81 16 with ADP case history 84 set()
82 83 17 a DET det history 84 set()
84 91 18 history NOUN nmod upregulated 45 set()
92 94 19 of ADP case reactivation 104 set()
95 103 20 repeated VERB amod reactivation 104 set()
104 116 

IntervalTree([Interval(22, 31, ('GENE_OR_GENE_PRODUCT', 'microRNA-')), Interval(60, 76, ('ORGANISM', 'murine Th1 cells')), Interval(124, 139, ('CELL', 'memory Th cells')), Interval(158, 166, ('TISSUE', 'synovial')), Interval(176, 184, ('ORGANISM', 'patients')), Interval(200, 205, ('MULTI-TISSUE_STRUCTURE', 'joint'))])

In [28]:
analyse(
(
    "A strong association existed between miR-494 and Th1 cytokines, and neurological deficit as measured by the National Institute of Health Stroke Scale (NIHSS) in AIS patients.",
    5,
    7
)
)

0 1 0 A DET det association 9 set()
2 8 1 strong ADJ amod association 9 set()
9 20 2 association NOUN nsubj existed 21 set()
21 28 3 existed VERB ROOT existed 21 set()
29 36 4 between ADP case miR-494 37 set()
37 44 5 miR-494 NOUN nmod existed 21 set()
45 48 6 and CCONJ cc miR-494 37 set()
49 52 7 Th1 NOUN conj miR-494 37 {('GENE_OR_GENE_PRODUCT', 'Th1')}
53 62 8 cytokines NOUN conj miR-494 37 set()
62 63 9 , PUNCT punct miR-494 37 set()
64 67 10 and CCONJ cc miR-494 37 set()
68 80 11 neurological ADJ amod deficit 81 set()
81 88 12 deficit NOUN conj miR-494 37 set()
89 91 13 as SCONJ mark measured 92 set()
92 100 14 measured VERB advcl existed 21 set()
101 103 15 by ADP case patients 165 set()
104 107 16 the DET det patients 165 set()
108 116 17 National PROPN compound Institute 117 set()
117 126 18 Institute PROPN dep the 104 set()
127 129 19 of ADP prep Institute 117 set()
130 136 20 Health PROPN compound Scale 144 set()
137 143 21 Stroke NOUN compound Scale 144 set()
144 149 22 Scal

IntervalTree([Interval(49, 52, ('GENE_OR_GENE_PRODUCT', 'Th1')), Interval(165, 173, ('ORGANISM', 'patients'))])

In [29]:
analyse(
(
    "We demonstrated that miR-494 inhibition prevented Th1-skewed neurotoxicity through regulation of the HDAC2-STAT4 cascade.",
    3,
    6
)
)

0 2 0 We PRON nsubj demonstrated 3 set()
3 15 1 demonstrated VERB ROOT demonstrated 3 set()
16 20 2 that SCONJ mark prevented 40 set()
21 28 3 miR-494 NOUN compound inhibition 29 set()
29 39 4 inhibition NOUN nsubj prevented 40 set()
40 49 5 prevented VERB ccomp demonstrated 3 set()
50 60 6 Th1-skewed ADJ amod neurotoxicity 61 {('GENE_OR_GENE_PRODUCT', 'Th1-skewed')}
61 74 7 neurotoxicity NOUN dobj prevented 40 set()
75 82 8 through ADP case regulation 83 set()
83 93 9 regulation NOUN nmod prevented 40 set()
94 96 10 of ADP case cascade 113 set()
97 100 11 the DET det cascade 113 set()
101 112 12 HDAC2-STAT4 NOUN amod cascade 113 {('GENE_OR_GENE_PRODUCT', 'HDAC2-STAT4')}
113 120 13 cascade NOUN nmod regulation 83 set()
120 121 14 . PUNCT punct demonstrated 3 set()
miR-494 Th1-skewed
miR-494 Th1-skewed
Conjunctions
SDP sent: We demonstrated that miR-494 inhibition prevented Th1-skewed neurotoxicity through regulation of the HDAC2-STAT4 cascade.
SDP     : [(miR-494, 'NOUN', 'compound', F

IntervalTree([Interval(50, 60, ('GENE_OR_GENE_PRODUCT', 'Th1-skewed')), Interval(101, 112, ('GENE_OR_GENE_PRODUCT', 'HDAC2-STAT4'))])